# Notebook for visualizations

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import row, layout
from bokeh.models import ColumnDataSource, Span, FactorRange
from bokeh.transform import factor_cmap, factor_mark
from bokeh.palettes import Spectral5, Viridis256, Colorblind, Magma256, Turbo256
from IPython.display import display, Math
from scipy import optimize
from math import exp, cos, sqrt, pi, sin, sqrt
from typing import List

from src.OspitalettoDataset import OspitalettoDataset
from src.NOAA2010Dataset import NOAA2010Dataset
from src.InsPireDataset import InsPireDataset

mypalette = sns.color_palette(['#51ffda']) # https://www.hsluv.org/
sns.set_palette(mypalette)

output_notebook()

Loading BokehJS ...

In [2]:
ospitaletto2019 = OspitalettoDataset()
noaa2010Dataset = NOAA2010Dataset()
insPireDataset = InsPireDataset()

AVAILABLE_DATASETS = dict()
AVAILABLE_DATASETS.update(ospitaletto2019.load_processed_data())
AVAILABLE_DATASETS.update(noaa2010Dataset.load_processed_data())
AVAILABLE_DATASETS.update(insPireDataset.load_processed_data())
AVAILABLE_DATASETS.keys()

dict_keys(['ospitaletto', 'miami_florida', 'fresno_california', 'olympia_washington', 'rochester_newyork', 'london_uk', 'madrid_spain', 'rome_italy', 'stuttgart_germany'])

## Visualization of curve fitting

In [3]:
ospitaletto = AVAILABLE_DATASETS[OspitalettoDataset.OSPITALETTO]

miami = AVAILABLE_DATASETS[NOAA2010Dataset.MIAMI_FL]
fresno = AVAILABLE_DATASETS[NOAA2010Dataset.FRESNO_CA]
olympia = AVAILABLE_DATASETS[NOAA2010Dataset.OLYMPIA_WA]
rochester = AVAILABLE_DATASETS[NOAA2010Dataset.ROCHESTER_NY]

london = AVAILABLE_DATASETS[InsPireDataset.LONDON_UK]
madrid = AVAILABLE_DATASETS[InsPireDataset.MADRID_SPA]
rome = AVAILABLE_DATASETS[InsPireDataset.ROME_IT]
stuttgart = AVAILABLE_DATASETS[InsPireDataset.STUTTGART_GER]

In [4]:
ospitaletto["city"] = "ospitaletto"

miami["city"] = "miami"
fresno["city"] = "fresno"
olympia["city"] = "olympia"
rochester["city"] = "rochester"
us_data = miami.append(other=fresno).append(other=olympia).append(other=rochester)

london["city"] = "london"
madrid["city"] = "madrid"
rome["city"] = "rome"
stuttgart["city"] = "stuttgart"
europe_data = london.append(other=madrid).append(other=rome).append(other=stuttgart)

In [5]:
source_usa = ColumnDataSource({"timestamp": miami.index,
                                 
                               "miami_air_temp": miami.air_temp,
                               "miami_air_temp_fit": miami.air_temp_fit,
                                 
                               "fresno_air_temp": fresno.air_temp,
                               "fresno_air_temp_fit": fresno.air_temp_fit,
                                 
                               "olympia_air_temp": olympia.air_temp,
                               "olympia_air_temp_fit": olympia.air_temp_fit,
                                 
                               "rochester_air_temp": rochester.air_temp,
                               "rochester_air_temp_fit": rochester.air_temp_fit,
                              })

color_map = {
    "miami": "navy",
    "fresno": "red",
    "olympia": "green",
    "rochester": "gray"
}

max_temp = int(us_data.air_temp.max() + 5)
min_temp = int(us_data.air_temp.min() - 5)

miami_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                x_axis_type="datetime",
                                title="Miami, Florida, USA",
                                y_range=(min_temp, max_temp),)
miami_air_temp_and_fit.line(x="timestamp", y="miami_air_temp", legend_label="Air Temp", color=color_map["miami"], alpha=0.2, source=source_usa,)
miami_air_temp_and_fit.circle(x="timestamp", y="miami_air_temp_fit", legend_label="it Air Temp", color=color_map["miami"], alpha=0.4, source=source_usa, size=1)


fresno_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                 x_axis_type="datetime",
                                 title="Fresno, California, USA",
                                 y_range=miami_air_temp_and_fit.y_range,
                                 x_range=miami_air_temp_and_fit.x_range)
fresno_air_temp_and_fit.line(x="timestamp", y="fresno_air_temp", legend_label="Air Temp", color=color_map["fresno"], alpha=0.2, source=source_usa)
fresno_air_temp_and_fit.circle(x="timestamp", y="fresno_air_temp_fit", legend_label="Fit Air Temp", color=color_map["fresno"], alpha=0.4, source=source_usa, size=1)

olympia_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                  x_axis_type="datetime",
                                  title="Olympia, Washington, USA",
                                  y_range=miami_air_temp_and_fit.y_range,
                                  x_range=miami_air_temp_and_fit.x_range)
olympia_air_temp_and_fit.line(x="timestamp", y="olympia_air_temp", legend_label="Air Temp", color=color_map["olympia"], alpha=0.2, source=source_usa)
olympia_air_temp_and_fit.circle(x="timestamp", y="olympia_air_temp_fit", legend_label="Fit Air Temp", color=color_map["olympia"], alpha=0.4, source=source_usa, size=1)

rochester_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                    x_axis_type="datetime",
                                    title="Rochester, New York, USA",
                                    y_range=miami_air_temp_and_fit.y_range,
                                    x_range=miami_air_temp_and_fit.x_range)
rochester_air_temp_and_fit.line(x="timestamp", y="rochester_air_temp", legend_label="Air Temp", color=color_map["rochester"], alpha=0.2, source=source_usa)
rochester_air_temp_and_fit.circle(x="timestamp", y="rochester_air_temp_fit", legend_label="Fit Air Temp", color=color_map["rochester"], alpha=0.4, source=source_usa, size=1)


only_air_temp_fit = figure(plot_width=1200, 
                           plot_height=300, 
                           x_axis_type="datetime",
                           title="Adjusted air temperature",
                           y_range=miami_air_temp_and_fit.y_range,
                           x_range=miami_air_temp_and_fit.x_range)
only_air_temp_fit.line(x="timestamp", y="miami_air_temp_fit", legend_label="Miami", color=color_map["miami"], alpha=0.7, source=source_usa, line_width=3)
only_air_temp_fit.line(x="timestamp", y="fresno_air_temp_fit", legend_label="Fresno", color=color_map["fresno"], alpha=0.7, source=source_usa, line_width=3)
only_air_temp_fit.line(x="timestamp", y="olympia_air_temp_fit", legend_label="Olympia", color=color_map["olympia"], alpha=0.7, source=source_usa, line_width=3)
only_air_temp_fit.line(x="timestamp", y="rochester_air_temp_fit", legend_label="Rochester", color=color_map["rochester"], alpha=0.7, source=source_usa, line_width=3)

show(layout([
    [miami_air_temp_and_fit, fresno_air_temp_and_fit,],
    [olympia_air_temp_and_fit, rochester_air_temp_and_fit],
    [only_air_temp_fit],
]))

In [6]:
source_europe = ColumnDataSource({"timestamp": london.index,
                                 
                                  "london_air_temp": london.air_temp,
                                  "london_air_temp_fit": london.air_temp_fit,
                                 
                                  "madrid_air_temp": madrid.air_temp,
                                  "madrid_air_temp_fit": madrid.air_temp_fit,
                                 
                                  "rome_air_temp": rome.air_temp,
                                  "rome_air_temp_fit": rome.air_temp_fit,
                                 
                                  "stuttgart_air_temp": stuttgart.air_temp,
                                  "stuttgart_air_temp_fit": stuttgart.air_temp_fit,
                                 })

color_map = {
    "london": "navy",
    "madrid": "red",
    "rome": "green",
    "stuttgart": "gray"
}

max_temp = int(europe_data.air_temp.max() + 5)
min_temp = int(europe_data.air_temp.min() - 5)

london_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                x_axis_type="datetime",
                                title="London, United Kingdom",
                                y_range=(min_temp, max_temp),)
london_air_temp_and_fit.line(x="timestamp", y="london_air_temp", legend_label="Air Temp", color=color_map["london"], alpha=0.2, source=source_europe)
london_air_temp_and_fit.circle(x="timestamp", y="london_air_temp_fit", legend_label="Fit Air Temp", color=color_map["london"], alpha=0.4, source=source_europe, size=1)

madrid_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                 x_axis_type="datetime",
                                title="Madrid, Spain",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
madrid_air_temp_and_fit.line(x="timestamp", y="madrid_air_temp", legend_label="Air Temp", color=color_map["madrid"], alpha=0.2, source=source_europe)
madrid_air_temp_and_fit.circle(x="timestamp", y="madrid_air_temp_fit", legend_label="Fit Air Temp", color=color_map["madrid"], alpha=0.4, source=source_europe, size=1)

rome_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                  x_axis_type="datetime",
                                title="Rome, Italy",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
rome_air_temp_and_fit.line(x="timestamp", y="rome_air_temp", legend_label="Air Temp", color=color_map["rome"], alpha=0.2, source=source_europe)
rome_air_temp_and_fit.circle(x="timestamp", y="rome_air_temp_fit", legend_label="Fit Air Temp", color=color_map["rome"], alpha=0.4, source=source_europe, size=1)

stuttgart_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                    x_axis_type="datetime",
                                title="Stuttgart, Germany",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
stuttgart_air_temp_and_fit.line(x="timestamp", y="stuttgart_air_temp", legend_label="Air Temp", color=color_map["stuttgart"], alpha=0.2, source=source_europe)
stuttgart_air_temp_and_fit.circle(x="timestamp", y="stuttgart_air_temp_fit", legend_label="Fit Air Temp", color=color_map["stuttgart"], alpha=0.4, source=source_europe, size=1)


only_air_temp_fit = figure(plot_width=1200, 
                           plot_height=300, 
                           x_axis_type="datetime",
                                title="Adjusted air temperature",
                          y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
only_air_temp_fit.line(x="timestamp", y="london_air_temp_fit", legend_label="London, UK", color=color_map["london"], alpha=0.7, source=source_europe, line_width=3)
only_air_temp_fit.line(x="timestamp", y="stuttgart_air_temp_fit", legend_label="Stuttgart, Germany", color=color_map["stuttgart"], alpha=0.7, source=source_europe, line_width=3)
only_air_temp_fit.line(x="timestamp", y="rome_air_temp_fit", legend_label="Rome, Italy", color=color_map["rome"], alpha=0.7, source=source_europe, line_width=3)
only_air_temp_fit.line(x="timestamp", y="madrid_air_temp_fit", legend_label="Madrid, Spain", color=color_map["madrid"], alpha=0.7, source=source_europe, line_width=3)

show(layout([
    [london_air_temp_and_fit, stuttgart_air_temp_and_fit ,],
    [rome_air_temp_and_fit, madrid_air_temp_and_fit],
    [only_air_temp_fit],
]))

## Visualization of Air Temp, Ground Temp and Aquifer Temp

In [9]:
source_usa = ColumnDataSource({"timestamp": miami.index,
                                 
                               "miami_air_temp": miami.air_temp,
                               "miami_ground_temp": miami.ground_temp,
                               "miami_aquifer_temp": miami.aquifer_temp,
                               
                               "fresno_air_temp": fresno.air_temp,
                               "fresno_ground_temp": fresno.ground_temp,
                               "fresno_aquifer_temp": fresno.aquifer_temp,
                                 
                               "olympia_air_temp": olympia.air_temp,
                               "olympia_ground_temp": olympia.ground_temp,
                               "olympia_aquifer_temp": olympia.aquifer_temp,
                                 
                               "rochester_air_temp": rochester.air_temp,
                               "rochester_ground_temp": rochester.ground_temp,
                               "rochester_aquifer_temp": rochester.aquifer_temp,
                              })

max_temp = int(us_data.air_temp.max() + 5)
min_temp = int(us_data.air_temp.min() - 5)

color_map = {
    "miami": "navy",
    "fresno": "red",
    "olympia": "green",
    "rochester": "gray"
}

miami_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                x_axis_type="datetime",
                                title="Miami, Florida, USA",
                                y_range=(min_temp, max_temp),)
miami_air_temp_and_fit.line(x="timestamp", y="miami_air_temp", legend_label="Air Temp", color=color_map["miami"], alpha=0.2, source=source_usa)
miami_air_temp_and_fit.circle(x="timestamp", y="miami_ground_temp", legend_label="Ground Temp", color=color_map["miami"], alpha=0.1, source=source_usa, size=1)


fresno_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                 x_axis_type="datetime",
                                title="Fresno, California, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
fresno_air_temp_and_fit.line(x="timestamp", y="fresno_air_temp", legend_label="Air Temp", color=color_map["fresno"], alpha=0.2, source=source_usa)
fresno_air_temp_and_fit.circle(x="timestamp", y="fresno_ground_temp", legend_label="Ground Temp", color=color_map["fresno"], alpha=0.1, source=source_usa, size=1)


olympia_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                  x_axis_type="datetime",
                                title="Olympia, Washington, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
olympia_air_temp_and_fit.line(x="timestamp", y="olympia_air_temp", legend_label="Air Temp", color=color_map["olympia"], alpha=0.2, source=source_usa)
olympia_air_temp_and_fit.circle(x="timestamp", y="olympia_ground_temp", legend_label="Ground Temp", color=color_map["olympia"], alpha=1, source=source_usa, size=1)


rochester_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                    x_axis_type="datetime",
                                title="Rochester, New York, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
rochester_air_temp_and_fit.line(x="timestamp", y="rochester_air_temp", legend_label="Air Temp", color=color_map["rochester"], alpha=0.2, source=source_usa)
rochester_air_temp_and_fit.circle(x="timestamp", y="rochester_ground_temp", legend_label="Ground Temp", color=color_map["rochester"], alpha=0.1, source=source_usa, size=1)


ground_temp = figure(plot_width=1200, 
                     plot_height=300, 
                     x_axis_type="datetime",
                     title="Estimated Ground Temperature",
                     y_range=london_air_temp_and_fit.y_range,
                     x_range=london_air_temp_and_fit.x_range)
ground_temp.line(x="timestamp", y="miami_ground_temp", legend_label="Miami, FL", color=color_map["miami"], alpha=0.7, source=source_usa, line_width=3)
ground_temp.line(x="timestamp", y="fresno_ground_temp", legend_label="Fresno, CA", color=color_map["fresno"], alpha=0.7, source=source_usa, line_width=3)
ground_temp.line(x="timestamp", y="olympia_ground_temp", legend_label="Olympia, WA", color=color_map["olympia"], alpha=0.7, source=source_usa, line_width=3)
ground_temp.line(x="timestamp", y="rochester_ground_temp", legend_label="Rochester, NY", color=color_map["rochester"], alpha=0.7, source=source_usa, line_width=3)


show(layout([
    [miami_air_temp_and_fit, fresno_air_temp_and_fit,],
    [olympia_air_temp_and_fit, rochester_air_temp_and_fit],
    [ground_temp],
]))

In [8]:
cities = ["Miami, FL", "Fresno, CA", "Olympia, WA", "Rochester, NY"]
stats = ['Aquifer Temp','Mean Air Temp']
mdata = {'cities' : cities,
         'Aquifer Temp': [miami.aquifer_temp.head(1).item(), 
                          fresno.aquifer_temp.head(1).item(),
                          olympia.aquifer_temp.head(1).item(), 
                          rochester.aquifer_temp.head(1).item()],
         'Mean Air Temp': [miami.air_temp.mean(), 
                           fresno.air_temp.mean(),
                           olympia.air_temp.mean(), 
                           rochester.air_temp.mean()],}

x = [(city, stat) for city in cities for stat in stats]
counts = sum(zip(mdata['Aquifer Temp'], mdata['Mean Air Temp']), ())

source = ColumnDataSource(data=dict(x=x, 
                                    counts=counts, 
                                    color=[color_map["miami"], color_map["miami"], 
                                                               color_map["fresno"], color_map["fresno"],
                                                               color_map["olympia"], color_map["olympia"], 
                                                               color_map["rochester"], color_map["rochester"]],
                                   alpha=[0.7, 0.4,
                                          0.7, 0.4,
                                          0.7, 0.4,
                                          0.7, 0.4,]))

aquifer_temp = figure(plot_width=1200, 
                      plot_height=600, 
                      x_range=FactorRange(*x), 
                      title="Comparison of aquifer and mean air temperatures across cities in the USA")
aquifer_temp.vbar(x='x', top='counts', fill_color='color', alpha="alpha", width=0.9, source=source)
aquifer_temp.y_range.start = 0
aquifer_temp.x_range.range_padding = 0.1
aquifer_temp.xaxis.major_label_orientation = .9
aquifer_temp.xgrid.grid_line_color = None

show(aquifer_temp)

In [11]:
source_europe = ColumnDataSource({"timestamp": london.index,
                                 
                                 "london_air_temp": london.air_temp,
                                 "london_ground_temp": london.ground_temp,
                                 "london_aquifer_temp": london.aquifer_temp,
                                 
                                 "stuttgart_air_temp": stuttgart.air_temp,
                                 "stuttgart_ground_temp": stuttgart.ground_temp,
                                 "stuttgart_aquifer_temp": stuttgart.aquifer_temp,
                                 
                                 "madrid_air_temp": madrid.air_temp,
                                 "madrid_ground_temp": madrid.ground_temp,
                                 "madrid_aquifer_temp": madrid.aquifer_temp,
                                 
                                 "rome_air_temp": rome.air_temp,
                                 "rome_ground_temp": rome.ground_temp,
                                 "rome_aquifer_temp": rome.aquifer_temp,
                                })

color_map = {
    "london": "navy",
    "madrid": "red",
    "rome": "green",
    "stuttgart": "gray"
}

max_temp = int(europe_data.air_temp.max() + 5)
min_temp = int(europe_data.air_temp.min() - 5)

london_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                x_axis_type="datetime",
                                title="London, United Kingdom",
                                y_range=(min_temp, max_temp),)
london_air_temp_and_fit.line(x="timestamp", y="london_air_temp", legend_label="Air Temp", color='navy', alpha=0.2, source=source_europe)
london_air_temp_and_fit.circle(x="timestamp", y="london_ground_temp", legend_label="Ground Temp", color='navy', alpha=0.1, source=source_europe, size=


stuttgart_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                 x_axis_type="datetime",
                                title="Stuttgart, Germany",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
stuttgart_air_temp_and_fit.line(x="timestamp", y="stuttgart_air_temp", legend_label="Air Temp", color='red', alpha=0.2, source=source_europe)
stuttgart_air_temp_and_fit.circle(x="timestamp", y="stuttgart_ground_temp", legend_label="Ground Temp", color='red', alpha=0.1, source=source_europe, size=1)


madrid_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                  x_axis_type="datetime",
                                title="Madrid, Spain",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
madrid_air_temp_and_fit.line(x="timestamp", y="madrid_air_temp", legend_label="Air Temp", color='green', alpha=0.2, source=source_europe)
madrid_air_temp_and_fit.circle(x="timestamp", y="madrid_ground_temp", legend_label="Ground Temp", color='green', alpha=1, source=source_europe, size=1)


rome_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                    x_axis_type="datetime",
                                title="Rome, Italy",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
rome_air_temp_and_fit.line(x="timestamp", y="rome_air_temp", legend_label="Air Temp", color='gray', alpha=0.2, source=source_europe)
rome_air_temp_and_fit.circle(x="timestamp", y="rome_ground_temp", legend_label="Ground Temp", color='gray', alpha=0.1, source=source_europe, size=1)


europe_ground_temp = figure(plot_width=1200, 
                            plot_height=300, 
                            x_axis_type="datetime",
                            title="Estimated Ground Temperature",
                            y_range=london_air_temp_and_fit.y_range,
                            x_range=london_air_temp_and_fit.x_range)
europe_ground_temp.line(x="timestamp", y="london_ground_temp", legend_label="London, UK", color=color_map["london"], alpha=0.7, source=source_europe, line_width=3)
europe_ground_temp.line(x="timestamp", y="stuttgart_ground_temp", legend_label="Stuttgart, Germany", color=color_map["stuttgart"], alpha=0.7, source=source_europe, line_width=3)
europe_ground_temp.line(x="timestamp", y="madrid_ground_temp", legend_label="Madrid, Spain", color=color_map["madrid"], alpha=0.7, source=source_europe, line_width=3)
europe_ground_temp.line(x="timestamp", y="rome_ground_temp", legend_label="Rome, Italy", color=color_map["rome"], alpha=0.7, source=source_europe, line_width=3)



show(layout([
    [london_air_temp_and_fit, stuttgart_air_temp_and_fit,],
    [madrid_air_temp_and_fit, rome_air_temp_and_fit],
    [europe_ground_temp],
]))

In [12]:
cities = ["London, UK", "Stuttgart, Germany", "Madrid, Spain", "Rome, Italy"]
stats = ['Aquifer Temp','Mean Air Temp']
mdata = {'cities' : cities,
         'Aquifer Temp': [london.aquifer_temp.head(1).item(), 
                          stuttgart.aquifer_temp.head(1).item(),
                          madrid.aquifer_temp.head(1).item(), 
                          rome.aquifer_temp.head(1).item()],
         'Mean Air Temp': [london.air_temp.mean(), 
                           stuttgart.air_temp.mean(),
                           madrid.air_temp.mean(), 
                           rome.air_temp.mean()],}

x = [(city, stat) for city in cities for stat in stats]
counts = sum(zip(mdata['Aquifer Temp'], mdata['Mean Air Temp']), ())

source = ColumnDataSource(data=dict(x=x, 
                                    counts=counts, 
                                    color=[color_map["london"], color_map["london"], 
                                           color_map["stuttgart"], color_map["stuttgart"],
                                           color_map["madrid"], color_map["madrid"], 
                                           color_map["rome"], color_map["rome"]],
                                   alpha=[0.7, 0.4,
                                          0.7, 0.4,
                                          0.7, 0.4,
                                          0.7, 0.4,]))

aquifer_temp = figure(plot_width=1200, 
                      plot_height=600, 
                      x_range=FactorRange(*x), 
                      title="Comparison of aquifer and mean air temperatures across cities in Europe")
aquifer_temp.vbar(x='x', top='counts', fill_color='color', alpha="alpha", width=0.9, source=source)
aquifer_temp.y_range.start = 0
aquifer_temp.x_range.range_padding = 0.1
aquifer_temp.xaxis.major_label_orientation = .9
aquifer_temp.xgrid.grid_line_color = None

show(aquifer_temp)

## Visualization of Hot Water and Space Heating temperatures

### Whole year

In [ ]:
visualize_on_same_row(data_list=[miami[["space_heating_temp","hot_water_temp"]],
                                 fresno[["space_heating_temp","hot_water_temp"]],
                                 olympia[["space_heating_temp","hot_water_temp"]],
                                 rochester[["space_heating_temp","hot_water_temp"]],
                                 ospitaletto[["space_heating_temp","hot_water_temp"]]],
                      ylabel="Temperature in Celsius", 
                      xlabel="Date",
                      suptitle='Calculated Space Heating and Hot Water temperatures')

In [ ]:
month = int(input("Enter month to analyze (values between 1 to 12): "))

In [ ]:
november_data = Tamb_hourly[Tamb_hourly.month == month ]
november_data[["space_heating_temp","hot_water_temp"]].plot(figsize=(10,6), color=["#4e97a2",'#82272e'])

plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

In [ ]:
Tamb_hourly["user_temp"].plot(figsize=(10,6))

In [ ]:
november_data = Tamb_hourly[Tamb_hourly.index.month == 11 ]
november_data["Tuser"].plot(figsize=(10,6))

## Visualization of Net temperature

In [ ]:
Tamb_hourly[["net_temp"]][Tamb_hourly.index < "2010-01-31"].plot()

## Visualization of COP

In [ ]:
Tamb_hourly[['COP']].plot(figsize=(20,6))

## Visualization of Energy Loss

In [ ]:
Tamb_hourly[['E_loss_tot']].plot(figsize=(20,6))

## Space Heating Distribution Plot

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(10,10))

# Stuttgart

sns.lineplot(data=stuttgart[["space_heating_dist"]], ax=ax1, color='#4d4dff')
ax1.set_title('Stuttgart')
ax1.set_ylabel('Space heating hourly distribution')
ax1.grid()

# London
sns.lineplot(data=london[["space_heating_dist"]], ax=ax2, color='#8686ef')
ax2.set_title('London')
ax2.grid()

# Madrid
sns.lineplot(data=madrid[["space_heating_dist"]], ax=ax3, color='#ef8e49')
ax3.set_title('Madrid')
ax3.set_ylabel('Space heating hourly distribution')
ax3.grid()

# Rome
sns.lineplot(data=rome[["space_heating_dist"]], ax=ax4, color= '#ca0c6e')
ax4.set_title('Rome')
ax4.grid()

fig.tight_layout()
plt.show()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(10,10))

# Stuttgart

sns.lineplot(data=stuttgart[["space_heating_dist"]], ax=ax1, color='#4d4dff')
ax1.set_title('Stuttgart')
ax1.set_ylabel('Space heating hourly distribution')
ax1.grid()

# London
sns.lineplot(data=london[["space_heating_dist"]], ax=ax2, color='#8686ef')
ax2.set_title('London')
ax2.grid()

# Madrid
sns.lineplot(data=madrid[["space_heating_dist"]], ax=ax3, color='#ef8e49')
ax3.set_title('Madrid')
ax3.set_ylabel('Space heating hourly distribution')
ax3.grid()

# Rome
sns.lineplot(data=rome[["space_heating_dist"]], ax=ax4, color= '#ca0c6e')
ax4.set_title('Rome')
ax4.grid()

fig.tight_layout()
plt.show()